In [ ]:
當然可以！以下是將程式碼中所有註解改為英文的版本。我已經確保註解清晰且準確地描述了程式碼的功能。完整的更新後程式碼如下所示，您可以直接複製並運行。
程式碼（英文註解版）
python
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from scipy.optimize import minimize
from scipy.special import expit
from scipy.interpolate import UnivariateSpline

# ===============================
# Interactive Dataset Visualization
# ===============================

# Define datasets (updated with new data)
datasets = {
    'Quantitative': {
        'color': 'red',
        'scale': np.array([
            90, 89, 88, 87, 86, 85, 84, 83, 82, 81,
            80, 79, 78, 77, 76, 75, 74, 73, 72, 71,
            70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60
        ]),
        'percentile': np.array([
            100, 97, 95, 94, 91, 88, 85, 81, 76, 70,
            64, 57, 50, 43, 37, 32, 26, 22, 19, 15,
            12, 10, 8, 6, 4, 3, 2, 2, 1, 1, 1
        ])
    },
    'Verbal': {
        'color': 'blue',
        'scale': np.array([
            90, 89, 88, 87, 86, 85, 84, 83, 82, 81,
            80, 79, 78, 77, 76, 75, 74, 73, 72, 71,
            70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60
        ]),
        'percentile': np.array([
            100, 99, 99, 98, 97, 94, 90, 84, 76, 67,
            57, 48, 39, 31, 23, 18, 13, 10, 7, 5,
            4, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1
        ])
    },
    'Data Insights': {
        'color': 'black',
        'scale': np.array([
            90, 89, 88, 87, 86, 85, 84, 83, 82, 81,
            80, 79, 78, 77, 76, 75, 74, 73, 72, 71,
            70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60
        ]),
        'percentile': np.array([
            100, 100, 99, 99, 99, 98, 97, 96, 93, 89,
            84, 77, 70, 63, 54, 48, 42, 36, 31, 26,
            21, 18, 15, 12, 10, 8, 7, 6, 5, 4, 4
        ])
    }
}

# Function: Find the corresponding percentile based on the scale score
def find_percentile(scale_score, dataset):
    """
    Calculate the percentile corresponding to the given scale score using interpolation.
    """
    scale = dataset['scale'][::-1]  # Reverse to ascending order
    percentile = dataset['percentile'][::-1]

    if scale_score < scale[0]:
        return percentile[0]
    elif scale_score > scale[-1]:
        return percentile[-1]
    else:
        return np.interp(scale_score, scale, percentile)

# Function: Calculate the estimated total score
def calculate_estimated_score(q, v, di):
    return -1005.3296 + 6.7098 * q + 6.6404 * v + 6.7954 * di

# Initialize plotting
plt.ioff()  # Turn off automatic display

# Define interactive sliders
q_slider = widgets.IntSlider(
    value=75,
    min=60,
    max=90,
    step=1,
    description='Quantitative (Q):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

v_slider = widgets.IntSlider(
    value=80,
    min=60,
    max=90,
    step=1,
    description='Verbal (V):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

di_slider = widgets.IntSlider(
    value=85,
    min=60,
    max=90,
    step=1,
    description='Data Insights (DI):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

# Output area for the score
score_output = widgets.Output()

# Function: Update plot and estimated score
def update_plot_and_score(q, v, di):
    with score_output:
        clear_output(wait=True)

        # Calculate estimated score
        estimated_score = calculate_estimated_score(q, v, di)

        # Update plot
        fig, ax = plt.subplots(figsize=(12, 8))

        for name, data in datasets.items():
            ax.scatter(
                data['scale'],
                data['percentile'],
                color=data['color'],
                label=name,
                alpha=0.7,
                edgecolors='w',
                s=100
            )

        # Candidate's scores and tangent lines
        candidate_scores = {
            'Quantitative': q,
            'Verbal': v,
            'Data Insights': di
        }

        for name, score in candidate_scores.items():
            dataset = datasets[name]
            percentile = find_percentile(score, dataset)

            # Reverse scale and percentile to ascending order
            sorted_scale = dataset['scale'][::-1]
            sorted_percentile = dataset['percentile'][::-1]

            # Use UnivariateSpline for smooth fitting
            try:
                spline = UnivariateSpline(sorted_scale, sorted_percentile, s=0.5)
            except Exception as e:
                print(f"Error fitting {name} data: {e}")
                continue

            # Calculate the slope (derivative) for the tangent
            slope = spline.derivative()(score)
            # Define the range for the tangent line (e.g., score ± 5)
            tangent_range = 5
            x_min = max(score - tangent_range, sorted_scale[0])
            x_max = min(score + tangent_range, sorted_scale[-1])
            x_tangent = np.linspace(x_min, x_max, 100)
            y_tangent = spline(score) + slope * (x_tangent - score)

            # Plot the tangent line
            ax.plot(
                x_tangent,
                y_tangent,
                color=dataset['color'],
                linestyle='--',
                linewidth=2,
                label=f"{name} Tangent at Scale {score}"
            )

            # Plot the candidate's score point
            ax.scatter(
                score,
                percentile,
                color=dataset['color'],
                edgecolors='w',
                s=200,
                marker='X',
                label=f"{name} Candidate"
            )

        # Customize the plot
        ax.set_xlabel('Scale', fontsize=14)
        ax.set_ylabel('Percentile', fontsize=14)
        ax.set_title('Scale vs. Percentile for Quantitative, Verbal, and Data Insights', fontsize=16)
        ax.grid(True, linestyle='--', alpha=0.5)

        ax.set_xlim(60, 90)
        ax.set_xticks(np.arange(60, 91, 5))

        ax.set_ylim(0, 100)
        ax.set_yticks(np.arange(0, 101, 10))

        # Update legend, removing duplicates
        handles, labels = ax.get_legend_handles_labels()
        by_label = OrderedDict(zip(labels, handles))
        ax.legend(by_label.values(), by_label.keys(), title='Groups', fontsize=12)

        plt.tight_layout()
        plt.show()

        # Display the estimated score
        print(f"Estimated Score: {estimated_score:.2f}")

# Connect sliders to the update function
widgets.interactive_output(
    update_plot_and_score,
    {
        'q': q_slider,
        'v': v_slider,
        'di': di_slider
    }
)

# Display sliders and output area
display(q_slider, v_slider, di_slider, score_output)

# Initial plot
update_plot_and_score(q_slider.value, v_slider.value, di_slider.value)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Cloning into 'GMAT-prompt'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 185 (delta 103), reused 78 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (185/185), 599.27 KiB | 10.33 MiB/s, done.
Resolving deltas: 100% (103/103), done.
所有 .md 檔案已合併到 combined_md_files.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>